In [102]:
import requests
import json
import pandas as pd

snapshot_api = 'https://hub.snapshot.org/graphql'
covalent_api = 'https://api.covalenthq.com/v1/'
covalent_key = 'ckey_1c9890fa556c457ab9ff050c2f4'

ens_token_addr = '0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198' # for testing purpose

# 1. Snapshot API
### (1) Query Proposals

In [65]:
GET_PROPOSALS = """
query ($first:Int!, $skip:Int!, $space_in:[String]!) {
  proposals (
    first: $first,
    skip: $skip,
    where: {
      space_in: $space_in,
    },
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    author
    space {
      id
      name
    }
  }
}
"""

In [67]:
r = requests.post(snapshot_api, json={
    'query': GET_PROPOSALS, 
    'variables': {
        'first': 1000,
        'skip': 0,
        'space_in': ['ens.eth']
    }
})
print('Status:', r.status_code)
# print(r.text)
# Load to JSON
json_data = json.loads(r.text)

Status: 200


In [68]:
# total # of proposals
proposals = json_data['data']['proposals']
len(proposals)

22

In [69]:
# Ids for each proposal
p0 = proposals[0]
p0.keys()
# dict_keys(['id', 'title', 'body', 'choices', 'start', 'end', 'snapshot', 'state', 'author', 'space'])

dict_keys(['id', 'title', 'body', 'choices', 'start', 'end', 'snapshot', 'state', 'author', 'space'])

In [70]:
for i in p0.keys():
    print('\n', i,':\n',p0[i])


 id :
 0x104eb11d42813fadc2b408856e8fa2c10e34dbb4a87abaa2f089ece124263f16

 title :
 [EP10] [Executable] Fund a DAO-Governed Identity Server

 body :
 Authors: [Gregory Rocco](https://github.com/obstropolos), [Wayne Chang](https://github.com/wyc)

# Description
This proposal is for the funding and establishment of a community-run OIDC Identity Provider Server for Sign-In with Ethereum, maintained by Spruce.

Temperature check originally from [this post](https://discuss.ens.domains/t/a-credibly-neutral-sign-in-with-ethereum-identity-provider-server/11001)

# Abstract
In our research, we found that many services wanted to integrate the Sign-In with Ethereum workflow but did not have the ability to add new passwordless authentication methods to their installations.

We also learned that most services already support OpenID Connect, and were open to adding a new Identity Provider that supported the SIWE workflow. By meeting those services where they are today, we can provide a pragmatic s

### 2. Get Votes

In [60]:
GET_VOTES = """query ($first:Int!, $skip:Int!, $proposal:String!) {
  votes (
    first: $first
    skip: $skip
    where: {
      proposal: $proposal
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    voter
    created
    proposal {
      id
    }
    choice
    space {
      id
    }
  }
}
"""

In [71]:
r = requests.post(snapshot_api, json={
    'query': GET_VOTES, 
    'variables': {
        'first': 1000,
        'skip': 0,
        'proposal': '0x104eb11d42813fadc2b408856e8fa2c10e34dbb4a87abaa2f089ece124263f16'
    }
})
print('Status:', r.status_code)
# print(r.text)
# Load to JSON
json_data = json.loads(r.text)

Status: 200


In [76]:
# total # of votes
votes = json_data['data']['votes']
len(votes)

445

In [79]:
v0 = votes[0]
for i in v0.keys():
    print('\n', i, ':\n', v0[i])


 id :
 0xb5502554b55912be5be38cbd7c3567decd324f8734a28e567848010c14028f58

 voter :
 0x6eB0F29b8Fc7A29E517BA56Dd79287B3CCaC64f3

 created :
 1647809547

 proposal :
 {'id': '0x104eb11d42813fadc2b408856e8fa2c10e34dbb4a87abaa2f089ece124263f16'}

 choice :
 1

 space :
 {'id': 'ens.eth'}


# 2. Covalent
### (1) Get all token holders

In [117]:
def r_token_holder_url(network_id, proposal_id, page_size, page_number):
    return covalent_api + str(network_id) + '/tokens/' + str(proposal_id) + '/token_holders/?quote-currency=USD&format=JSON&page-size=' + str(page_size) + '&page-number=' + str(page_number) + '&key=' + covalent_key

request_url = r_token_holder_url(1, ens_token_addr, 10000, 0)
print(request_url)
r = requests.get(request_url)
print('Status:', r.status_code)
# Load to JSON
json_data = json.loads(r.text)

https://api.covalenthq.com/v1/1/tokens/0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198/token_holders/?quote-currency=USD&format=JSON&page-size=10000&page-number=0&key=ckey_1c9890fa556c457ab9ff050c2f4
Status: 200


In [120]:
json_data['data'].keys()

dict_keys(['updated_at', 'items', 'pagination'])

In [119]:
json_data['data']['pagination']

{'has_more': False, 'page_number': 0, 'page_size': 10000, 'total_count': 4872}

In [121]:
len(json_data['data']['items'])

4872

In [115]:
request_url = r_token_holder_url(1, ens_token_addr, 1000, 1)
print(request_url)
r = requests.get(request_url)
print('Status:', r.status_code)
# Load to JSON
json_data = json.loads(r.text)

https://api.covalenthq.com/v1/1/tokens/0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198/token_holders/?quote-currency=USD&format=JSON&page-size=1000&page-number=1&key=ckey_1c9890fa556c457ab9ff050c2f4
Status: 200


In [116]:
json_data

{'data': {'updated_at': '2022-04-09T01:47:11.928585165Z',
  'items': [],
  'pagination': {'has_more': False,
   'page_number': 1,
   'page_size': 1000,
   'total_count': 1000}},
 'error': False,
 'error_message': None,
 'error_code': None}